In [1]:
import pandas as pd                #for data manipulation
import numpy as np
pd.set_option('max_columns', 300)
pd.set_option('max_rows', 5000)
import matplotlib.pyplot as plt
import warnings
from datetime import datetime
warnings.simplefilter('ignore')

In [2]:
data = pd.read_csv('/Users/martinacoyne/Downloads/owid-covid-data-25.csv')
deaths3 = pd.read_csv('/Users/martinacoyne/Downloads/Provisional_COVID-19_Deaths_by_Week__Sex__and_Age-5.csv')

In [3]:
import sqlite3 
conn = sqlite3.connect('covid1.db')
def run_query(q):
    with sqlite3.connect('covid1.db') as conn:
        return pd.read_sql(q, conn)

def run_command(c):
    with sqlite3.connect('covid1.db') as conn:
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        conn.execute(c)

In [4]:
weekly = data[data['location'].isin(['United States','Italy','France','United Kingdom','Sweden','Germany','Mexico',
                                     'Belgium','Spain', 'Australia', 'Kenya', 'Israel'])][['date','location','total_cases','new_cases','total_deaths','new_deaths','new_cases_smoothed', 'new_deaths_smoothed', 'hosp_patients', 'icu_patients', 'total_tests','new_tests','positive_rate']]
weekly = weekly.fillna(0)
for each in range(len(weekly)):
    date = weekly.date.iloc[each]
    weekly.date.iloc[each] = datetime.strptime(date, "%Y-%m-%d")

In [5]:
run_command(
'''
DROP TABLE if exists age
'''
)

deaths3.to_sql('age', conn)

age_deaths = run_query(
'''
WITH number_one AS (

SELECT
    --"MMWR Week",
    "Age Group" Age_Group,
    SUM("COVID-19 Deaths") NumDeaths
FROM age
WHERE Sex LIKE '%ALL%'
GROUP BY "Age Group"
--AND "MMWR Week" = 48
)

SELECT 
    *,
    ROUND((CAST(NumDeaths AS float)/672020)*100,3) PercentDeaths
FROM number_one
ORDER BY NumDeaths


    
''')

age_deaths[['NumDeaths','PercentDeaths']] = age_deaths[['NumDeaths','PercentDeaths']].astype(float)
age_deaths = pd.DataFrame(age_deaths.iloc[1]).transpose().append([pd.DataFrame(age_deaths.iloc[0]).transpose(), 
                                                                  age_deaths.iloc[2:-1]])

In [6]:
from bokeh.palettes import Category20c
from bokeh.plotting import figure, show
from bokeh.transform import cumsum
from bokeh.layouts import row, column
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.models import BoxAnnotation
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.layouts import gridplot, layout
from bokeh.models import LinearAxis, Range1d
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn


# prepare some weekly
v = list(weekly[weekly['location'] == 'United States'].new_cases.iloc[-570:])
w = list(weekly[weekly.location == 'United States'].iloc[-570:].new_cases_smoothed)
x = list(weekly[weekly['location'] == 'United States'].date.iloc[-570:])
y = list(weekly[weekly['location'] == 'United States'].new_deaths.iloc[-570:])
z = list(weekly[weekly.location == 'United States'].iloc[-570:].new_deaths_smoothed)


# create a new plot with a title and axis labels
p = figure(title="COVID-19 Deaths", x_axis_label="Date", y_axis_label="Number of Deaths", plot_height=500, 
           plot_width=1200, x_axis_type="datetime")
q = figure(title='COVID-19 Cases', x_axis_label="Date", y_axis_label="Number of Cases", plot_height=500, 
           plot_width=1200, x_axis_type="datetime")

# add a line renderer with legend and line thickness
#p.line(x, y, legend_label="Temp.", line_width=2)
p.vbar(x=x, top=y, bottom=0, color='blue', alpha=.1, width=1.0)
p.line(x=x, y=z, legend_label="7 Day Average", width=1.0, color="red")

q.vbar(x=x, top=v, bottom=0, color='green', alpha=.1, width=1.0)
q.line(x=x, y=w, legend_label="7 Day Average", width=1.0, color="red")

p.legend.location = "top_left"
q.legend.location = "top_left"


## United States ##

x = list(weekly[weekly.location == "United States"].date.iloc[-570:])
y = list(weekly[weekly.location == "United States"].iloc[-570:].new_cases_smoothed)
y2 = list(weekly[weekly.location == "United States"].iloc[-570:].new_deaths_smoothed)

us = figure(title="United States", plot_height=300, plot_width=600, x_axis_type="datetime")

us.line(x, y, color="blue", legend_label="7 Day Case Average", alpha=0.45)
us.extra_y_ranges = {"foo": Range1d(start=-10, end=weekly[weekly.location == "United States"]
                                    .iloc[-570:].new_deaths_smoothed.max() + 100)}
us.line(x, y2, color="orange", y_range_name="foo", legend_label="7 Day Death Average")
us.add_layout(LinearAxis(y_range_name="foo"), 'right')
us.xaxis.ticker.desired_num_ticks = 15
us.legend.location = "top_left"


## Italy ##

a = list(weekly[weekly.location == "Italy"].iloc[-570:].new_cases_smoothed)
a2 = list(weekly[weekly.location == "Italy"].iloc[-570:].new_deaths_smoothed)

ita_dates = list(weekly[weekly.location == "Italy"].iloc[-570:].date)

ita = figure(title="Italy", plot_height=300, plot_width=600, x_axis_type="datetime")

ita.line(x, a, color="blue", legend_label="7 Day Case Average", alpha=0.45)

ita.extra_y_ranges = {"ita": Range1d(start=-10, end=weekly[weekly.location == "Italy"].iloc[-570:]
                                     .new_deaths_smoothed.max() + 100)}
ita.line(x, a2, color="orange", y_range_name="ita", legend_label="7 Day Death Average")
ita.add_layout(LinearAxis(y_range_name="ita"), 'right')
ita.xaxis.ticker.desired_num_ticks = 15


## United Kingdom ##

b = list(weekly[weekly.location == "United Kingdom"].iloc[-570:].new_cases_smoothed)
b2 = list(weekly[weekly.location == "United Kingdom"].iloc[-570:].new_deaths_smoothed)

uk = figure(title="United Kingdom", plot_height=300, plot_width=600, x_axis_type="datetime")

uk.line(x, b, color="blue", legend_label="7 Day Case Average", alpha=0.45)

uk.extra_y_ranges = {"uk": Range1d(start=-10, end=weekly[weekly.location == "United Kingdom"].iloc[-570:]
                                   .new_deaths_smoothed.max() + 100)}
uk.line(x, b2, color="orange", y_range_name="uk", legend_label="7 Day Death Average")
uk.add_layout(LinearAxis(y_range_name="uk"), 'right')
uk.xaxis.ticker.desired_num_ticks = 15
uk.legend.location = "top_left"



## France ##

c = list(weekly[weekly.location == "France"].iloc[-570:].new_cases_smoothed)
c2 = list(weekly[weekly.location == "France"].iloc[-570:].new_deaths_smoothed)

fra = figure(title="France", plot_height=300, plot_width=600, x_axis_type="datetime")

fra.line(x, c, color="blue", legend_label="7 Day Case Average", alpha=0.45)

fra.extra_y_ranges = {"fra": Range1d(start=-10, end=weekly[weekly.location == "France"].iloc[-570:]
                                     .new_deaths_smoothed.max() + 100)}
fra.line(x, c2, color="orange", y_range_name="fra", legend_label="7 Day Death Average")
fra.add_layout(LinearAxis(y_range_name="fra"), 'right')
fra.xaxis.ticker.desired_num_ticks = 15
fra.legend.glyph_height = 10
fra.legend.label_text_font_size = '10px'

## Sweden ##

d = list(weekly[weekly.location == "Sweden"].iloc[-570:].new_cases_smoothed)
d2 = list(weekly[weekly.location == "Sweden"].iloc[-570:].new_deaths_smoothed)

sw = figure(title="Sweden", plot_height=300, plot_width=600, x_axis_type="datetime")

sw.line(x, d, color="blue", legend_label="7 Day Case Average", alpha=0.45)

sw.extra_y_ranges = {"swe": Range1d(start=-10, end=weekly[weekly.location == "Sweden"].iloc[-570:]
                                    .new_deaths_smoothed.max() + 10)}
sw.line(x, d2, color="orange", y_range_name="swe", legend_label="7 Day Death Average")
sw.add_layout(LinearAxis(y_range_name="swe"), 'right')
sw.xaxis.ticker.desired_num_ticks = 15
sw.legend.location = "top_left"



## Germany ##

e = list(weekly[weekly.location == "Germany"].iloc[-570:].new_cases_smoothed)
e2 = list(weekly[weekly.location == "Germany"].iloc[-570:].new_deaths_smoothed)

ger = figure(title="Germany", plot_height=300, plot_width=600, x_axis_type="datetime")

ger.line(x, e, color="blue", legend_label="7 Day Case Average", alpha=0.45)

ger.extra_y_ranges = {"ger": Range1d(start=-10, end=weekly[weekly.location == "Germany"].iloc[-570:]
                                     .new_deaths_smoothed.max() + 100)}
ger.line(x, e2, color="orange", y_range_name="ger", legend_label="7 Day Death Average")
ger.add_layout(LinearAxis(y_range_name="ger"), 'right')
ger.xaxis.ticker.desired_num_ticks = 15
ger.legend.location = "top_left"



## Mexico

j = list(weekly[weekly.location == "Mexico"].iloc[-570:].new_cases_smoothed)
j2 = list(weekly[weekly.location == "Mexico"].iloc[-570:].new_deaths_smoothed)

mex = figure(title="Mexico", plot_height=300, plot_width=600, x_axis_type="datetime")

mex.line(x, j, color="blue", legend_label="7 Day Case Average", alpha=0.45)

mex.extra_y_ranges = {"mex": Range1d(start=-10, end=weekly[weekly.location == "Mexico"].iloc[-570:]
                                     .new_deaths_smoothed.max() + 100)}
mex.line(x, j2, color="orange", y_range_name="mex", legend_label="7 Day Death Average")
mex.add_layout(LinearAxis(y_range_name="mex"), 'right')
mex.xaxis.ticker.desired_num_ticks = 15
mex.legend.location = "top_left"



## Belgium ##

k = list(weekly[weekly.location == "Belgium"].iloc[-570:].new_cases_smoothed)
k2 = list(weekly[weekly.location == "Belgium"].iloc[-570:].new_deaths_smoothed)

bel = figure(title="Belgium", plot_height=300, plot_width=600, x_axis_type="datetime")

bel.line(x, k, color="blue", legend_label="7 Day Case Average", alpha=0.45)

bel.extra_y_ranges = {"bel": Range1d(start=-1, end=weekly[weekly.location == "Belgium"].iloc[-570:]
                                     .new_deaths_smoothed.max() + 10)}
bel.line(x, k2, color="orange", y_range_name="bel", legend_label="7 Day Death Average")
bel.add_layout(LinearAxis(y_range_name="bel"), 'right')
bel.xaxis.ticker.desired_num_ticks = 15


## Australia ##

l = list(weekly[weekly.location == "Australia"].iloc[-570:].new_cases_smoothed)
l2 = list(weekly[weekly.location == "Australia"].iloc[-570:].new_deaths_smoothed)

aus = figure(title="Australia", plot_height=300, plot_width=600, x_axis_type="datetime")

aus.line(x, l, color="blue", legend_label="7 Day Case Average", alpha=0.45)

aus.extra_y_ranges = {"aus": Range1d(start=-.5, end=weekly[weekly.location == "Australia"].iloc[-570:]
                                     .new_deaths_smoothed.max() + 2)}
aus.line(x, l2, color="orange", y_range_name="aus", legend_label="7 Day Death Average")
aus.add_layout(LinearAxis(y_range_name="aus"), 'right')
aus.xaxis.ticker.desired_num_ticks = 15
aus.legend.location = "top_left"



## Kenya ##

m = list(weekly[weekly.location == "Kenya"].iloc[-570:].new_cases_smoothed)
m2 = list(weekly[weekly.location == "Kenya"].iloc[-570:].new_deaths_smoothed)

ken = figure(title="Kenya", plot_height=300, plot_width=600, x_axis_type="datetime")

ken.line(x, m, color="blue", legend_label="7 Day Case Average", alpha=0.45)

ken.extra_y_ranges = {"ken": Range1d(start=-1, end=weekly[weekly.location == "Kenya"].iloc[-570:]
                                     .new_deaths_smoothed.max() + 10)}
ken.line(x, m2, color="orange", y_range_name="ken", legend_label="7 Day Death Average")
ken.add_layout(LinearAxis(y_range_name="ken"), 'right')
ken.xaxis.ticker.desired_num_ticks = 15
ken.legend.location = "top_left"



## Israel ##

n = list(weekly[weekly.location == "Israel"].iloc[-570:].new_cases_smoothed)
n2 = list(weekly[weekly.location == "Israel"].iloc[-570:].new_deaths_smoothed)

isr = figure(title="Israel", plot_height=300, plot_width=600, x_axis_type="datetime")

isr.line(x, n, color="blue", legend_label="7 Day Case Average", alpha=0.45)

isr.extra_y_ranges = {"isr": Range1d(start=-1, end=weekly[weekly.location == "Israel"].iloc[-570:]
                                     .new_deaths_smoothed.max() + 10)}
isr.line(x, n2, color="orange", y_range_name="isr", legend_label="7 Day Death Average")
isr.add_layout(LinearAxis(y_range_name="isr"), 'right')
isr.xaxis.ticker.desired_num_ticks = 15
isr.legend.location = "top_left"


## Data Table ##

source = ColumnDataSource(weekly[weekly.location == 'United States'].iloc[-15:])

columns = [
        TableColumn(field="date", title="Date", formatter=DateFormatter()),
        TableColumn(field="new_cases", title="New Cases"),
        TableColumn(field="new_deaths", title="New Deaths"),
        TableColumn(field="new_cases_smoothed", title="Cases 7-day"),
        TableColumn(field="new_deaths_smoothed", title="Deaths 7-day"),
    ]
data_table = DataTable(source=source, columns=columns, width=600, height=300)

## Deaths Data ##

pie_deaths = age_deaths[['Age_Group', 'PercentDeaths']]

pie_deaths['angle'] = pie_deaths['PercentDeaths']/pie_deaths['PercentDeaths'].sum() * 2 * np.pi
pie_deaths['color'] = Category20c[len(pie_deaths)]

pie = figure(height=500, width=600, title="Covid-19 Deaths Breakdown by Age Group", toolbar_location=None,
           tools="hover", tooltips="@Age_Group: @PercentDeaths", x_range=(-0.5, 1.0))

pie.wedge(x=0.055, y=10, radius=0.45,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='Age_Group', source=pie_deaths)

pie.legend.location = "top_right"
pie.axis.axis_label = None
pie.axis.visible = False
pie.grid.grid_line_color = None
pie.title.align = "center"

## Data Table ##

source = ColumnDataSource(age_deaths)

columns = [TableColumn(field=x, title=x) for x in age_deaths.columns]

deaths_table = DataTable(source=source, columns=columns, width=600, height=1000, fit_columns = True, 
                         index_width = 60, height_policy = 'fit', row_height=43)


#show(column(children=[p, isr]))
#grid = gridplot([[p], 
#             [q], [us, ita], [uk, fra], [sw, ger], [mex, bel], [aus, ken], [isr]], sizing_mode="scale_width")
#show(grid)

first_layout = layout([[p], [q], [us, data_table], [uk, sw], [mex, ken], [bel, ger], [isr, aus], [pie, deaths_table]], sizing_mode="fixed", max_height=200)
show(first_layout)